<a href="https://colab.research.google.com/github/disola/naive-bayes/blob/main/naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementierung von Naive Bayes:

Die Aufgabe ist es, gegeben eines Feedbacks an die Direkt Bahn (DB) zu entscheiden, ob das Feedback ein positives oder ein negatives Sentiment hat.
Dazu implementieren wir die Naive Bayes-Methode.

Die Implementierung umfasst die folgenden Schritte:

A. Daten vorbereiten:
1. Lade den annotierten Datensatz als Dataframe
2. Bereinige den Datensatz

B. Naive-Bayes-Modell trainieren
3. Zähle die Worthäufigkeiten je Klasse
4. Berechne Log Prior und die Log-Likelihood-Werte λ je Wort

C. Naives-Bayes-Modell anwenden
5. Nutze das Modell zur Sentimentanalyse

Wir nutzen den kompletten Datensatz zum Training und wenden das Modell auf beliebigen von Nutzenden eingegebenen Feedbacks an. Aufgrund der beschränkten Zeit machen wir keinen Train-Test-Split und beschränken uns auf das Vokabular, das durch den Datensatz gegeben ist.

## A. Daten vorbereiten

### 1. Lade den annotierten Datensatz als Dataframe

In [1]:
feedbacks = [
    ("Es lief gut, Zug pünktlich.", 1),
    ("Fahrt war gut.", 1),
    ("Es lief schlecht, Zug verspätet.", 0),
    ("Fahrt war nicht gut.", 0),
    ("Alles wie geplant, super Service.", 1),
    ("Sehr angenehme Reise, alles top.", 1),
    ("Pünktlich und sauber, gerne wieder.", 1),
    ("Zug war ruhig und komfortabel.", 1),
    ("Nettes Personal und gute Verbindung.", 1),
    ("Schnelle Fahrt, keine Probleme.", 1),
    ("Bequeme Sitze und gutes WLAN.", 1),
    ("Reise verlief ohne Zwischenfälle.", 1),
    ("Zug kam sogar früher an.", 1),
    ("Einwandfreie Reiseerfahrung.", 1),
    ("Sehr zufrieden mit der Fahrt.", 1),
    ("Klasse Verbindung, keine Umstiege.", 1),
    ("Alles wie geplant, super Service.", 1),
    ("Zugfahrt war sehr angenehm.", 1),
    ("Personal war freundlich und hilfsbereit.", 1),
    ("Fahrtzeit wie angekündigt, super.", 1),
    ("Verbindung war ideal, danke.", 1),
    ("Toller Ausblick während der Fahrt.", 1),
    ("Reise verlief schlecht.", 0),
    ("Klimaanlage war kaputt.", 0),
    ("WLAN war kaputt.", 0),
    ("Zug hatte über eine Stunde Verspätung.", 0),
    ("Klima im Waggon war schlecht.", 0),
    ("Toiletten waren nicht sauber.", 0),
    ("Unfreundliches Personal.", 0),
    ("Zug ist ausgefallen.", 0),
    ("Musste lange auf Anschluss warten.", 0),
    ("Überfüllt und laut.", 0),
    ("Keine Klimaanlage bei Hitze.", 0),
    ("Zug kam gar nicht.", 0),
    ("Reise war sehr unangenehm.", 0),
    ("WLAN hat nicht funktioniert.", 0),
    ("Sitze waren kaputt.", 0),
    ("Zug hielt mitten auf der Strecke.", 0),
    ("Ständige Verspätungen.", 0),
    ("Musste im Gang stehen.", 0)
]

Wir nutzen *pandas*, da es eine effiziente Datenverarbeitung erlaubt.

In [2]:
import pandas as pd

In [3]:
feedback_dict = {
    "text": [f[0] for f in feedbacks],
    "label": [f[1] for f in feedbacks]
}

In [4]:
df = pd.DataFrame(feedback_dict)
# Zeige die ersten vier Zeilen des Dataframes an
df.head(4)

,text,label
0,"Es lief gut, Zug pünktlich.",1
1,Fahrt war gut.,1
2,"Es lief schlecht, Zug verspätet.",0
3,Fahrt war nicht gut.,0


In [5]:
# Bestimme die Anzahl der Feedbacks
len(df)

40

In [6]:
# Zähle, wie oft jeder eindeutige Wert in der Spalte 'label' des DataFrames df vorkommt.
df["label"].value_counts()

,count
label,
1,20
0,20


Der Datensatz ist ausgeglichen: Er enthält jeweils 20 als positiv bzw. negativ annotierte Feedbacks.

### 2. Bereinige den Datensatz

Wir nutzen *spacy*, das eine schnelle Textverarbeitung erlaubt. Eine Alternative wäre *nltk* - ein Package, das flexibler, aber langsamer ist.

In [7]:
import spacy
!python -m spacy download de_core_news_sm
nlp = spacy.load("de_core_news_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 73.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
def preprocess(text):
    """
    Verarbeitet einen gegebenen Text mit einem NLP-Modell:
    - Wandelt den Text in Kleinbuchstaben um
    - Entfernt Satzzeichen und Leerzeichen
    - Gibt eine Liste von Wörtern zurück ("Tokenisierung")

    Parameter:
    text (str): Der Eingabetext, der verarbeitet werden soll.

    Rückgabe:
    list: Eine Liste von Wörtern (Tokens) ohne Satz- und Leerzeichen.
    """
    doc = nlp(text.lower()) # wandle in Kleinbuchstaben um
    return [
        token.text # gebe eine Liste von Wörtern zurück
        for token in doc
        if not token.is_punct and not token.is_space  # entferne Leer- und Satzzeichen
    ]

Wende das Pre-Processing auf das Dataframe an.

In [9]:
# Erstelle eine neue Spalte mit dem Namen "tokens", die sich aus der Anwendung der "preprocess" Funktion auf die "text"-Spalte ergibt.
df["tokens"] = df["text"].apply(preprocess)

In [10]:
df.head(4)

,text,label,tokens
0,"Es lief gut, Zug pünktlich.",1,"[es, lief, gut, zug, pünktlich]"
1,Fahrt war gut.,1,"[fahrt, war, gut]"
2,"Es lief schlecht, Zug verspätet.",0,"[es, lief, schlecht, zug, verspätet]"
3,Fahrt war nicht gut.,0,"[fahrt, war, nicht, gut]"


## B. Naive-Bayes-Modell trainieren

### 3. Zähle die Worthäufigkeiten je Klasse

In [11]:
def create_freq_table(token_lists, labels):
    """
    Zählt die Häufigkeit jedes Wortes (Tokens) in Kombination mit einem zugehörigen Label (0 oder 1)
    und gibt ein Dictionary mit diesen Wort-Label-Paaren als Schlüssel zurück.

    Parameter:
    token_lists (list of list of str): Eine Liste, in der jede Eintragung eine Liste von Tokens (Wörtern) ist.
    labels (list of int): Eine Liste von Labels (0 oder 1), jeweils entsprechend einer Token-Liste.

    Rückgabe:
    dict: Ein Dictionary mit (Wort, Label)-Tupeln als Schlüssel und deren Häufigkeit als Wert.
    """
    result = {}
    for tokens, y in zip(token_lists, labels):
        for word in tokens:
            pair = (word, y)
            if pair in result.keys():
                result[pair] += 1
            else:
                result[pair] = 1
    return result

In [12]:
# Erstelle die Worthäufigkeiten-Tabelle als Dictionary aus dem Dataframe df
freq = create_freq_table(df['tokens'], df['label'])

In [13]:
# Zeige die ersten Einträge an
for pair in list(freq.items())[:10]:
    print(pair)

(('es', 1), 1)
(('lief', 1), 1)
(('gut', 1), 2)
(('zug', 1), 3)
(('pünktlich', 1), 2)
(('fahrt', 1), 4)
(('war', 1), 5)
(('es', 0), 1)
(('lief', 0), 1)
(('schlecht', 0), 3)


Um die Einträge aus dem Dictionary mit den Häufigkeiten gut abrufen zu können, schreiben wir auch eine Look-up-Funktion.

In [14]:
def lookup(freq, word, label):
    """
    Gibt die Häufigkeit eines Wort-Label-Paares aus einem Frequenz-Dictionary zurück.

    Parameter:
    freq (dict): Ein Dictionary mit (Wort, Label)-Tupeln als Schlüssel und deren Häufigkeit als Wert.
    word (str): Das Wort, nach dem gesucht werden soll.
    label (int): Das zugehörige Label (z. B. 0 oder 1), das mit dem Wort kombiniert wird.

    Rückgabe:
    int: Die Anzahl der Vorkommen des (Wort, Label)-Paares im Dictionary. Gibt 0 zurück, wenn das Paar nicht vorhanden ist.
    """
    n = 0
    pair = (word, label)
    if pair in freq:
        n = freq[pair]
    return n

In [15]:
# Wende die Lookup Funktion an, um die Häufigkeit des Wortes "es" mit dem Label "1" zu erhalten
lookup(freq,"es",1)

1

### 4. Berechne Log Prior und die Log-Likelihood-Werte λ je Wort


Wir nutzen *numpy* für die Log-Berechnungen

In [16]:
import numpy as np

In [17]:
def train_naive_bayes(freq, train_x, train_y):
    """
    Trainiert ein Naive-Bayes-Modell zur Sentiment-Analyse auf Basis von Wortfrequenzen.

    Parameter:
    freq (dict): Ein Dictionary mit (Wort, Label)-Tupeln als Schlüssel und ihrer Häufigkeit als Wert.
    train_x (list of str): Eine Liste von Texten in Form von Token-Listen
    train_y (list of int): Eine Liste von zugehörigen Labels (0 = negativ, 1 = positiv).

    Rückgabe:
    logprior (float): Der logarithmische Prior-Wert (Verhältnis positiver zu negativer Texte).
    loglikelihood (dict): Ein Dictionary mit Wörtern als Schlüssel und deren log-Likelihood-Werten als Wert.

    """
    loglikelihood = {}
    logprior = 0

    # Vokabular bestimmen (alle eindeutigen Wörter in freq)
    vocab = set([pair[0] for pair in freq.keys()])
    n = len(vocab)  # Anzahl der eindeutigen Wörter im Vokabular

    # Zähler für positive und negative Wortvorkommen initialisieren
    # N_pos und N_neg sind jeweils die Summen der Spalten der Wortfrequenztabelle
    N_pos = N_neg = 0
    for pair in freq.keys():
        if pair[1] > 0:
            # Wenn das Label positiv ist, erhöhe Zähler für positive Wörter
            N_pos += freq[pair]
        else:
            # Wenn das Label negativ ist, erhöhe Zähler für negative Wörter
            N_neg += freq[pair]

    # Gesamtanzahl der Texte
    D = len(train_y)

    # Anzahl der positiven Texte
    D_pos = len([y for y in train_y if y > 0])

    # Anzahl der negativen Texte (alle übrigen)
    D_neg = D - D_pos

    # Berechne den logprior (logarithmisches Verhältnis positiver zu negativer Texte)
    logprior = np.log(D_pos / D_neg)

    # Für jedes Wort im Vokabular...
    for word in vocab:
        # Häufigkeit des Wortes bei positiven und negativen Labels abrufen
        freq_pos = lookup(freq, word, 1)
        freq_neg = lookup(freq, word, 0)

        # Wahrscheinlichkeiten mit Laplace-Smoothing berechnen
        p_w_pos = (freq_pos + 1) / (N_pos + n)
        p_w_neg = (freq_neg + 1) / (N_neg + n)

        # Log-Likelihood berechnen und speichern
        loglikelihood[word] = np.log(p_w_pos / p_w_neg)

    return logprior, loglikelihood


In [18]:
logprior, loglikelihood = train_naive_bayes(freq, df['tokens'], df['label'])
print(logprior)
print(len(loglikelihood))

0.0
96


## C. Naive-Bayes-Modell anwenden

### 5. Nutze das Modell zur Sentimentanalyse

In [19]:
def naive_bayes_predict(text, logprior, loglikelihood):
    """
    Schätzt die Wahrscheinlichkeit, dass ein gegebener Text (z.B. Feedback) positiv oder negativ ist,
    basierend auf einem trainierten Naive-Bayes-Modell.

    Parameter:
    text (str): Der Eingabetext, der klassifiziert werden soll.
    logprior (float): Der logarithmische Prior-Wert aus dem Training (log(P(positiv) / P(negativ))).
    loglikelihood (dict): Ein Dictionary, das jedem Wort einen Log-Likelihood-Wert zuordnet.

    Rückgabe:
    float: Die geschätzte Wahrscheinlichkeit (in Log-Werten), dass der Text zur positiven Klasse gehört.
           Ein positiver Wert spricht für eine positive, ein negativer für eine negative Klasse.
    """

    # Den Text vorverarbeiten und in einzelne Wörter (Tokens) zerlegen
    tokens = preprocess(text)

    # Wahrscheinlichkeitswert initialisieren
    p = 0

    # Logprior-Wert zur Gesamtwahrscheinlichkeit addieren
    p += logprior

    # Für jedes Wort im Text:
    for word in tokens:
        # Wenn das Wort im Log-Likelihood-Wörterbuch vorhanden ist,
        if word in loglikelihood:
            # füge den Log-Likelihood-Wert des Wortes zur Gesamtwahrscheinlichkeit hinzu
            p += loglikelihood[word]

    return p

In [20]:
test_text = 'Fahrt lief gut.'

In [21]:
# Wende das Modell auf des Test-Text an
p = naive_bayes_predict(test_text, logprior, loglikelihood)
sentiment = "positiv" if p > 0 else "negativ" if p < 0 else "neutral"
print(f"Das Sentiment von \"{test_text}\" ist {sentiment} (p={p:.2f}).")

Das Sentiment von "Fahrt lief gut." ist positiv.
